# PyTorch Cifar10 local training  

## Pre-requisites

This notebook shows how to use the SageMaker Python SDK to run your code in a local container before deploying to SageMaker's managed training or hosting environments.  This can speed up iterative testing and debugging while using the same familiar Python SDK interface.  Just change your estimator's `train_instance_type` to `local` (or `local_gpu` if you're using an ml.p2 or ml.p3 notebook instance).

In order to use this feature you'll need to install docker-compose (and nvidia-docker if training with a GPU).

**Note, you can only run a single local notebook at one time.**

In [1]:
!/bin/bash ./setup.sh

/bin/bash: ./setup.sh: No such file or directory


## Overview

The **SageMaker Python SDK** helps you deploy your models for training and hosting in optimized, productions ready containers in SageMaker. The SageMaker Python SDK is easy to use, modular, extensible and compatible with TensorFlow, MXNet, PyTorch and Chainer. This tutorial focuses on how to create a convolutional neural network model to train the [Cifar10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) using **PyTorch in local mode**.

### Set up the environment

This notebook was created and tested on a single ml.p2.xlarge notebook instance.

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the sagemaker.get_execution_role() with appropriate full IAM role arn string(s).

In [2]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-opennre'

role = sagemaker.get_execution_role()

In [3]:
import os
import subprocess

instance_type = 'local'

if subprocess.call('nvidia-smi') == 0:
    ## Set type to GPU if one is present
    instance_type = 'local_gpu'
    
print("Instance type = " + instance_type)

Instance type = local_gpu


### Upload the data
We use the ```sagemaker.Session.upload_data``` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use this later when we start the training job.

In [4]:
data_inputs = sagemaker_session.upload_data(path='../example/benchmark/FinRE', bucket=bucket, key_prefix='data/FinRE')
pretrain_inputs = sagemaker_session.upload_data(path='../example/pretrain/tencent', bucket=bucket, key_prefix='pretrain/tencent')
inputs = {'training': data_inputs, 'pretrain': pretrain_inputs}
print(inputs)

{'training': 's3://sagemaker-us-east-1-579019700964/data/FinRE', 'pretrain': 's3://sagemaker-us-east-1-579019700964/pretrain/tencent'}


## Script Functions

SageMaker invokes the main function defined within your training script for training. When deploying your trained model to an endpoint, the model_fn() is called to determine how to load your trained model. The model_fn() along with a few other functions list below are called to enable predictions on SageMaker.

### [Predicting Functions](https://github.com/aws/sagemaker-pytorch-containers/blob/master/src/sagemaker_pytorch_container/serving.py)
* model_fn(model_dir) - loads your model.
* input_fn(serialized_input_data, content_type) - deserializes predictions to predict_fn.
* output_fn(prediction_output, accept) - serializes predictions from predict_fn.
* predict_fn(input_data, model) - calls a model on data deserialized in input_fn.

The model_fn() is the only function that doesn't have a default implementation and is required by the user for using PyTorch on SageMaker. 

## Create a training job using the sagemaker.PyTorch estimator

The `PyTorch` class allows us to run our training function on SageMaker. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type. For local training with GPU, we could set this to "local_gpu".  In this case, `instance_type` was set above based on your whether you're running a GPU instance.

After we've constructed our `PyTorch` object, we fit it using the data we uploaded to S3. Even though we're in local mode, using S3 as our data source makes sense because it maintains consistency with how SageMaker's distributed, managed training ingests data.


In [10]:
from sagemaker.pytorch import PyTorch

#git_config = {'repo': 'https://github.com/whn09/OpenNRE.git', 'branch': 'master'}

cifar10_estimator = PyTorch(entry_point='example/train_finre_cnn_softmax.py',  # TODO bug
                            source_dir='.',
                            #git_config=git_config,
                            role=role,
                            framework_version='1.4.0',
                            train_instance_count=1,
                            train_instance_type=instance_type)

cifar10_estimator.fit(inputs)

Creating tmpzugroy_algo-1-9lkag_1 ... 
Attaching to tmpzugroy_algo-1-9lkag_12mdone
algo-1-9lkag_1  | 2020-04-17 03:21:59,368 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
algo-1-9lkag_1  | 2020-04-17 03:21:59,395 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
algo-1-9lkag_1  | 2020-04-17 03:21:59,398 sagemaker_pytorch_container.training INFO     Invoking user training script.
algo-1-9lkag_1  | 2020-04-17 03:22:02,420 sagemaker-containers INFO     Installing module with the following command:
algo-1-9lkag_1  | /opt/conda/bin/python -m pip install . -r requirements.txt
algo-1-9lkag_1  | Processing /tmp/tmpzi8x83fx/module_dir
algo-1-9lkag_1  | Requirement already satisfied: wheel in /opt/conda/lib/python3.6/site-packages (from -r requirements.txt (line 1)) (0.34.2)
algo-1-9lkag_1  | Collecting nltk==3.4.5
algo-1-9lkag_1  |   Downloading nltk-3.4.5.zip (1.5 MB)
     |████████████████████████████████| 1.5 MB 15

RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpzUgROY/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1

# Deploy the trained model to prepare for predictions

The deploy() method creates an endpoint (in this case locally) which serves prediction requests in real-time.

In [ ]:
from sagemaker.pytorch import PyTorchModel

cifar10_predictor = cifar10_estimator.deploy(initial_instance_count=1,
                                             instance_type=instance_type)

# Invoking the endpoint

In [ ]:
# get some test images
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%4s' % classes[labels[j]] for j in range(4)))

outputs = cifar10_predictor.predict(images.numpy())

_, predicted = torch.max(torch.from_numpy(np.array(outputs)), 1)

print('Predicted: ', ' '.join('%4s' % classes[predicted[j]]
                              for j in range(4)))

# Clean-up

Deleting the local endpoint when you're finished is important since you can only run one local endpoint at a time.

In [ ]:
cifar10_estimator.delete_endpoint()